# Ex 3-1

In [7]:
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
from dataset import load_hoda

In [8]:
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda()
x_train = x_train_original.astype('float32')
x_test = x_test_original.astype('float32')
x_train /= 255
x_test /= 255

In [18]:
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model
import datetime
import os

logDir = ".\\logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(logDir, histogram_freq=1)

In [19]:
model = Sequential()
model.add(Dense(60, activation='relu', input_dim=25))
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu', input_dim=25))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(x_train, y_train_original,
                    epochs=100, batch_size=64, 
                    validation_split=0.2,
                    verbose=1,
                    callbacks=[tensorboard_callback])

Epoch 1/100
13/13 [==============================] - 1s 23ms/step - loss: 2.2099 - accuracy: 0.2138 - val_loss: 2.0279 - val_accuracy: 0.6250
Epoch 2/100
13/13 [==============================] - 0s 8ms/step - loss: 1.9442 - accuracy: 0.4963 - val_loss: 1.7390 - val_accuracy: 0.7450
Epoch 3/100
13/13 [==============================] - 0s 9ms/step - loss: 1.6779 - accuracy: 0.6050 - val_loss: 1.3655 - val_accuracy: 0.7900
Epoch 4/100
13/13 [==============================] - 0s 9ms/step - loss: 1.3173 - accuracy: 0.6988 - val_loss: 1.0026 - val_accuracy: 0.8450
Epoch 5/100
13/13 [==============================] - 0s 8ms/step - loss: 1.0361 - accuracy: 0.7275 - val_loss: 0.7466 - val_accuracy: 0.8600
Epoch 6/100
13/13 [==============================] - 0s 8ms/step - loss: 0.8232 - accuracy: 0.7800 - val_loss: 0.6085 - val_accuracy: 0.8600
Epoch 7/100
13/13 [==============================] - 0s 9ms/step - loss: 0.7074 - accuracy: 0.7950 - val_loss: 0.5217 - val_accuracy: 0.8700
Epoch 8/100


# Ex 3-2

In [22]:
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
from dataset import load_hoda

In [23]:
X_train, y_train, X_test, y_test = load_hoda(training_sample_size=3500,
                                             test_sample_size=400,size=28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = (X_train - 127.5) / 127.5
X_test = (X_test - 127.5) / 127.5
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [33]:
def get_learning_rate_from_file(epoch):
    filename = "lr.txt"
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.split('#', 1)[0]
            if line:
                par = line.strip().split(':')
                e = int(par[0])
                lr = float(par[1])
                if e <= epoch and lr > 0:
                    learning_rate = lr
                else:
                    return learning_rate
                    return learning_rate
        return learning_rate

learning_rate_from_file_scheduler = LearningRateScheduler(get_learning_rate_from_file, verbose=1)

In [34]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])
history = model.fit(X_train, y_train,
          epochs=200, batch_size=256, validation_split=0.2,
          callbacks=[learning_rate_from_file_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/200
11/11 [==============================] - 3s 168ms/step - loss: 0.0191 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.0760 - val_sparse_categorical_accuracy: 0.9757 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/200
11/11 [==============================] - 2s 151ms/step - loss: 0.0202 - sparse_categorical_accuracy: 0.9961 - val_loss: 0.0763 - val_sparse_categorical_accuracy: 0.9800 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/200
11/11 [==============================] - 2s 164ms/step - loss: 0.0180 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.0671 - val_sparse_categorical_accuracy: 0.9814 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/200
11/11 [==============================] - 2s 151ms/step - loss: 0.0124 - sparse_categorical_accuracy: 0.9982 - val_loss: 0.0598 - val_sparse_cate

# Ex 3-3

In [35]:
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
from dataset import load_hoda

In [36]:
X_train, y_train, X_test, y_test = load_hoda(training_sample_size=3500,
                                             test_sample_size=400,size=28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = (X_train - 127.5) / 127.5
X_test = (X_test - 127.5) / 127.5
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [40]:
import tensorflow as tf

class StopTrainingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get('val_sparse_categorical_accuracy')
        if val_accuracy is not None and val_accuracy >= 0.75:
            print("\nReached 75% validation accuracy. Stopping training.")
            self.model.stop_training = True

In [41]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])
history = model.fit(X_train, y_train,
                    epochs=200, 
                    batch_size=256, 
                    validation_split=0.2,
                    callbacks=[StopTrainingCallback()])

Epoch 1/200
11/11 [==============================] - 3s 161ms/step - loss: 0.5478 - sparse_categorical_accuracy: 0.8189 - val_loss: 1.9065 - val_sparse_categorical_accuracy: 0.7186
Epoch 2/200
11/11 [==============================] - 2s 144ms/step - loss: 0.2979 - sparse_categorical_accuracy: 0.9232 - val_loss: 1.8873 - val_sparse_categorical_accuracy: 0.2314
Epoch 3/200
11/11 [==============================] - 2s 147ms/step - loss: 0.1874 - sparse_categorical_accuracy: 0.9507 - val_loss: 1.9859 - val_sparse_categorical_accuracy: 0.1714
Epoch 4/200
11/11 [==============================] - 2s 146ms/step - loss: 0.1423 - sparse_categorical_accuracy: 0.9664 - val_loss: 2.1006 - val_sparse_categorical_accuracy: 0.1700
Epoch 5/200
11/11 [==============================] - 2s 147ms/step - loss: 0.1088 - sparse_categorical_accuracy: 0.9782 - val_loss: 2.2849 - val_sparse_categorical_accuracy: 0.1629
Epoch 6/200
11/11 [==============================] - 2s 146ms/step - loss: 0.0832 - sparse_cate

# Ex 4-2

In [ ]:
import os, shutil

original_dataset_dir = 'D:/dataset/catDog/train'
base_dir = 'D:/dataset/catDog/catVsdog'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# Directory with our validation cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# Directory with our validation dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)